In [2]:
search_for = 135
start_from = 0
threads = 10
thread = 3
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.01651930809020996
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 175
all cases: 723027151
type: [1, 1, 1, 1, 135] 135
cases of this type: 332150625
100000 39.486803134306854
200000 67.87675992890333
300000 66.21932934643397
400000 65.40214939253455
500000 65.08075132275012
600000 66.00579745622193
700000 73.09080964877927
800000 72.37274870452627
900000 68.45307762172841
1000000 75.1333708127966
1100000 68.81458149978239
1200000 62.551774296262515
1300000 25.294635773274237
1400000 70.56162699573126
1500000 74.13132395632489
1600000 69.76547475176994
1700000 70.43993766963419
1800000 70.50972015409111
1900000 72.44754325957089
2000000 68.10558396326594
2100000 70.2622656076376
2200000 69.47695035823585
2300000 68.67112177695296
2400000 71.51729375745475
2500000 22.411949155183255
2600000 66.3299277341161
2700000 87.58957714168686
2800000 89.76192141649423
2900000 86.62065298666012
3000000 87.9962443095596
3

30100000 74.70278960617256
30200000 70.39428198816637
30300000 67.08681249122645
30400000 71.28559308832111
30500000 70.34373322160313
30600000 69.51793154988408
30700000 68.74482957544686
30800000 15.258745895131744
30900000 73.35272014449967
31000000 69.4211637635515
31100000 70.54227988180006
31200000 72.63613818737313
31300000 68.78286615479139
31400000 69.63444397168163
31500000 76.95954407345735
31600000 70.95578113519367
31700000 73.62606350301121
31800000 25.39241127852393
31900000 79.29532297413665
32000000 71.30127466379852
32100000 75.98337561236532
32200000 73.0709214963823
32300000 31.52817622050554
32400000 74.71769827325622
32500000 73.59208664892665
32600000 75.43605231305257
32700000 71.69221086501287
32800000 72.69293985399335
32900000 72.79568325151368
33000000 68.37792074863695
33100000 67.98518759948531
33200000 53.14302697144936
33300000 36.96041220201377
33400000 65.97575175080239
33500000 65.40347991673572
33600000 70.65203756585208
33700000 70.63511297837492
33

60500000 63.66289487002343
60600000 64.84338345339383
60700000 61.53763038979653
60800000 66.36233436224457
60900000 63.90854991027789
61000000 66.95450629994782
61100000 28.212854532189606
61200000 76.65150870287353
61300000 70.74170403852675
61400000 69.49580590063273
61500000 70.11963941078463
61600000 71.79252808858425
61700000 73.83810931634952
61800000 74.30091453031685
61900000 72.72091931514831
62000000 35.690673870640175
62100000 64.17682054124849
62200000 70.35304261039984
62300000 69.07022650346597
62400000 66.96775691313132
62500000 65.12475667731802
62600000 63.87813899624664
62700000 63.70007593918087
62800000 19.01035619937024
62900000 65.69034367105708
63000000 65.75748582128722
63100000 67.87139393650232
63200000 64.14003364491694
63300000 71.56250696194914
63400000 67.93270381724956
63500000 52.8984540832425
63600000 50.20294852204633
63700000 71.83106728113736
63800000 73.34433025552073
63900000 70.6144670941796
64000000 71.67507798591647
64100000 71.36908536085716
6

90900000 63.74747615481673
91000000 63.618138272416275
91100000 61.10817960917028
91200000 66.58471861520746
91300000 67.10900615759557
91400000 64.72953376920076
91500000 60.147714196035594
91600000 63.374712966838004
91700000 48.77917017483541
91800000 33.053168651588635
91900000 54.73397752992684
92000000 58.44559759473306
92100000 57.49026711650321
92200000 57.54431177277394
92300000 18.095714664424833
92400000 56.9225368812102
92500000 57.29837340156263
92600000 56.9852947296473
92700000 60.18407739951106
92800000 43.5346623403015
92900000 38.604520213132055
93000000 60.18480637269373
93100000 57.22878003225002
93200000 60.809511758102246
93300000 63.21815342817389
93400000 61.01150861294411
93500000 65.10914125804221
93600000 67.46130173959126
93700000 65.63045127118633
93800000 63.32603695587214
93900000 64.79982059442575
94000000 60.38269129991793
94100000 67.22400485409045
94200000 37.89908974216486
94300000 41.66642893177376
94400000 57.39316601278861
94500000 60.385363721838

120400000 63.720979662443966
120500000 62.07392357179472
120600000 62.08972443502186
120700000 62.929417667641175
120800000 66.735348579425
120900000 70.97424601842093
121000000 62.33465911687265
121100000 60.92543406331353
121200000 60.72103403221488
121300000 55.685614205223814
121400000 60.50355825435849
121500000 22.37931374148859
121600000 56.64289411886444
121700000 55.19028755287509
121800000 33.173372473693874
121900000 42.86461487400285
122000000 59.97260826694497
122100000 49.21433334192471
122200000 38.388809290640815
122300000 57.99634715222624
122400000 58.88758970175487
122500000 54.98342767449175
122600000 63.645304393867484
122700000 60.00306669249811
122800000 67.56307908342296
122900000 67.72292441407555
123000000 69.83398290576406
123100000 65.82251446740116
123200000 67.57309462876442
123300000 67.06657580954167
123400000 66.53539516612815
123500000 62.51979328075168
123600000 61.30273182788501
123700000 59.93853035570108
123800000 67.32459528027502
123900000 59.261

...

640300000 2.025486848439717
640400000 1.8439637172546048
640500000 1.8027772191069034
640600000 1.7802170075235904
640700000 1.7193442604561482
640800000 1.7932292379284804
640900000 1.9562979086646315
641000000 2.2093787041802355
641100000 1.9925828307358626
641200000 1.772830985169774
641300000 1.7155806500375597
641400000 1.7030511406599196
641500000 1.5940502561072927
641600000 1.699942149527492
641700000 1.8388330691840207
641800000 2.023620652347261
641900000 2.03024801644855
642000000 1.8930641724121564
642100000 1.7168671304348333
642200000 1.6288194266335017
642300000 1.5210771064818693
642400000 1.5674712644852895
642500000 1.671664758207216
642600000 1.813905776602523
642700000 1.964645693576914
642800000 1.845904773139153
642900000 1.7678657897806167
643000000 1.6659395922351143
643100000 1.5274720748115893
643200000 1.4816278932231282
643300000 1.5197636925754587
643400000 1.6064554143254666
643500000 1.6807463775204905
643600000 1.729298809369886
643700000 1.6832698119327

667800000 1.510166933378997
667900000 0.9437491725319803
668000000 0.935525215501876
668100000 0.9538615948838266
668200000 0.9366521518842816
668300000 0.9359332863237572
668400000 0.9517213104379079
668500000 0.942816878865395
668600000 0.9485443541273906
668700000 0.9603264315436086
668800000 0.9828561008808195
668900000 0.9643738003422817
669000000 0.956347157561028
669100000 0.9449206796130141
669200000 0.9609232477944718
669300000 1.0368547661624663
669400000 1.05292424432019
669500000 0.9362394045613626
669600000 1.0069519496156851
669700000 0.9570386246753334
669800000 1.1035718204730147
669900000 1.1322510330800812
670000000 1.0566881401269774
670100000 1.02866005799301
670200000 1.0204462855309526
670300000 1.0895642114809434
670400000 1.0876477668023408
670500000 1.0480601836768566
670600000 1.0232490308538733
670700000 1.0024975119088186
670800000 0.9537568854909494
670900000 0.9388302398808638
671000000 0.9308293209791124
671100000 0.9683430088435591
671200000 0.9140569866

692600000 1.4340355942376546
692700000 2.297963588352184
692800000 1.1848653819627384
692900000 1.73205002921661
693000000 1.901767922585549
693100000 1.4491726058066188
693200000 1.8963698761255812
693300000 1.6951019790210697
693400000 1.2714195527523557
693500000 2.093121972624237
693600000 1.3262263616931982
693700000 1.3872491841869337
693800000 1.8306600589615338
693900000 1.5410865047687061
694000000 1.578942090607935
694100000 1.840267769970519
694200000 1.4156847187892225
694300000 1.6862191705738088
694400000 1.7632191273128197
694500000 1.5705280377648958
694600000 1.3082201016040722
694700000 1.750135258654884
694800000 1.8219367791536158
694900000 1.4768812729814291
695000000 1.660257114622999
695100000 1.7562008412657024
695200000 1.5238643571288526
695300000 1.8319017321455904
695400000 1.3211316009583405
695500000 2.0891650536078017
695600000 1.732727668134447
695700000 1.480297396830199
695800000 1.4979799083471
695900000 2.0531894348779494
best so far: 0
type: [1, 3, 

best so far: 0
type: [1, 9, 3, 5, 1] 135
cases of this type: 10125
713300000 0.09151354339330263
best so far: 0
type: [1, 9, 5, 1, 3] 135
cases of this type: 18225
best so far: 0
type: [1, 9, 5, 3, 1] 135
cases of this type: 6075
best so far: 0
type: [1, 9, 15, 1, 1] 135
cases of this type: 2025
best so far: 0
type: [1, 15, 1, 1, 9] 135
cases of this type: 98415
713400000 0.10830578539277712
best so far: 0
type: [1, 15, 1, 3, 3] 135
cases of this type: 32805
best so far: 0
type: [1, 15, 1, 9, 1] 135
cases of this type: 10935
best so far: 0
type: [1, 15, 3, 1, 3] 135
cases of this type: 10935
best so far: 0
type: [1, 15, 3, 3, 1] 135
cases of this type: 3645
best so far: 0
type: [1, 15, 9, 1, 1] 135
cases of this type: 1215
best so far: 0
type: [1, 27, 1, 1, 5] 135
cases of this type: 16875
713500000 0.11088043100306591
best so far: 0
type: [1, 27, 1, 5, 1] 135
cases of this type: 3375
best so far: 0
type: [1, 27, 5, 1, 1] 135
cases of this type: 675
best so far: 0
type: [1, 45, 1, 1, 3

best so far: 0
type: [9, 1, 1, 15, 1] 135
cases of this type: 3375
723000000 0.00031653902460800277
best so far: 0
type: [9, 1, 3, 1, 5] 135
cases of this type: 5625
best so far: 0
type: [9, 1, 3, 5, 1] 135
cases of this type: 1125
best so far: 0
type: [9, 1, 5, 1, 3] 135
cases of this type: 2025
best so far: 0
type: [9, 1, 5, 3, 1] 135
cases of this type: 675
best so far: 0
type: [9, 1, 15, 1, 1] 135
cases of this type: 225
best so far: 0
type: [9, 3, 1, 1, 5] 135
cases of this type: 1875
best so far: 0
type: [9, 3, 1, 5, 1] 135
cases of this type: 375
best so far: 0
type: [9, 3, 5, 1, 1] 135
cases of this type: 75
best so far: 0
type: [9, 5, 1, 1, 3] 135
cases of this type: 405
best so far: 0
type: [9, 5, 1, 3, 1] 135
cases of this type: 135
best so far: 0
type: [9, 5, 3, 1, 1] 135
cases of this type: 45
best so far: 0
type: [9, 15, 1, 1, 1] 135
cases of this type: 15
best so far: 0
type: [15, 1, 1, 1, 9] 135
cases of this type: 6561
best so far: 0
type: [15, 1, 1, 3, 3] 135
cases of